In [1]:
import os
import dask.dataframe as dd
from dask.distributed import Client, LocalCluster
import multiprocessing
import datetime
import logging
import numpy as np

# Configurar o logging para visualizar os logs de informação
logging.basicConfig(level=logging.INFO)

In [2]:
raw_dataset_path = '../datasets/BTCUSDT-Trades/'
output_base_path = '../output'
timestamp = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
output_path_versioned = f'{output_base_path}_v{timestamp}'

In [3]:
# 1. Configuração do Cluster Dask com ajustes para otimização de memória
def setup_dask_cluster():
    num_cores = multiprocessing.cpu_count()
    print(f"Número de núcleos disponíveis: {num_cores}")

    # Definindo número de workers e threads por worker
    n_workers = 3  # Reduzido para 3 workers
    threads_per_worker = 4  # Ajustado para 4 threads por worker
    memory_limit = '16GB'  # 8 GB por worker

    cluster = LocalCluster(
        n_workers=n_workers,
        threads_per_worker=threads_per_worker,
        memory_limit=memory_limit
    )

    client = Client(cluster)
    print(client)
    return client

# 2. Leitura dos Arquivos Parquet com otimização de memória
def read_parquet_files(raw_dataset_path):
    parquet_pattern = os.path.join(raw_dataset_path, 'BTCUSDT-Dataset-part-*.parquet')
    df_dask = dd.read_parquet(
        parquet_pattern,
        columns=['price', 'qty', 'quoteQty', 'time'],
        engine='pyarrow'
    )

    # Otimizar tipos de dados para reduzir uso de memória
    df_dask['price'] = df_dask['price'].astype('float32')
    df_dask['qty'] = df_dask['qty'].astype('float32')
    return df_dask

# 3. Aplicação das Operações Matemáticas
def apply_operations(df_dask):
    # Calcular 'trade_dollar' = price * qty
    df_dask['trade_dollar'] = df_dask['price'] * df_dask['qty']

    # Calcular 'dollar_side' = -1 se isBuyerMaker for True, 1 se False
    df_dask['dollar_side'] = df_dask['isBuyerMaker'].map({True: -1, False: 1}, meta=('dollar_side', 'int8'))

    # Calcular 'dollar_imbalance' = trade_dollar * dollar_side
    df_dask['dollar_imbalance'] = df_dask['trade_dollar'] * df_dask['dollar_side']

    return df_dask

# 4. Salvar o DataFrame Processado
def save_processed_dataframe(df_dask, output_path):
    os.makedirs(output_path, exist_ok=True)
    try:
        df_dask.to_parquet(
            output_path,
            write_index=False,
            compression='snappy',
            row_group_size=500 * 1024 * 1024,  # 500MB em bytes
            overwrite=True
        )
        print(f"DataFrame processado salvo com sucesso em: {output_path}")
    except Exception as e:
        print(f"Erro ao salvar o DataFrame: {e}")




In [4]:
# Liste todos os arquivos e pastas no diretório
files = os.listdir(raw_dataset_path)

# Filtre apenas os arquivos
file_count = sum(1 for f in files if os.path.isfile(os.path.join(raw_dataset_path, f)))

for number in range(1, file_count):
    print(number)
    file = 'BTCUSDT-Dataset-part-' + str(number) + '.parquet'
    df_dask = dd.read_parquet(os.path.join(raw_dataset_path, file))
    df_dask.head()

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37


In [5]:
df_dask.head()

,trade_id,price,qty,quoteQty,time,isBuyerMaker,isBestMatch
0,3782520000,57802.64,0.0346,1999.971344,2024-09-02 08:14:09.678,True,True
1,3782520001,57802.07,0.0001,5.780207,2024-09-02 08:14:09.679,True,True
2,3782520002,57802.07,0.0001,5.780207,2024-09-02 08:14:09.679,True,True
3,3782520003,57802.07,0.0001,5.780207,2024-09-02 08:14:09.679,True,True
4,3782520004,57802.07,0.0001,5.780207,2024-09-02 08:14:09.679,True,True


In [6]:
# 5. Função Principal para Orquestrar o Processo
def main():

    # Configuração do Cluster
    client = setup_dask_cluster()

    try:
        # Leitura dos Arquivos
        df_dask = read_parquet_files(raw_dataset_path)
        print("Arquivos Parquet lidos com sucesso.")

        # Aplicação das Operações Matemáticas
        df_dask = apply_operations(df_dask)
        print("Operações matemáticas aplicadas com sucesso.")

        # Opcional: Persistir os dados na memória para otimizar múltiplas operações
        df_dask = df_dask.persist()
        logging.info("DataFrame persistido na memória.")

        # Cálculo de Métricas
        media_preco = df_dask['price'].mean()
        total_dollar_imbalance = df_dask['dollar_imbalance'].sum()
        df_filtrado = df_dask[df_dask['qty'] > 1000]

        # Executar o processamento com tratamento de erros
        try:
            resultado_media_preco = media_preco.compute()
            resultado_total_dollar_imbalance = total_dollar_imbalance.compute()
            resultado_df_filtrado = df_filtrado.compute()

            logging.info(f"Média do preço: {resultado_media_preco}")
            logging.info(f"Total do dollar imbalance: {resultado_total_dollar_imbalance}")
            logging.info("DataFrame filtrado (exemplo):")
            logging.info(resultado_df_filtrado.head())
        except Exception as compute_error:
            print(f"Erro durante a computação das métricas: {compute_error}")



        # Salvar o DataFrame Processado
        save_processed_dataframe(df_dask, output_path_versioned)

        # # Validação dos Dados Salvos
        # try:
        #     df_reloaded = dd.read_parquet(output_path_versioned)
        #     media_preco_reloaded = df_reloaded['price'].mean().compute()
        #     total_dollar_imbalance_reloaded = df_reloaded['dollar_imbalance'].sum().compute()

        #     print(f"Média do preço após o salvamento: {media_preco_reloaded}")
        #     print(f"Total do dollar imbalance após o salvamento: {total_dollar_imbalance_reloaded}")
        # except Exception as validation_error:
        #     print(f"Erro ao validar os dados salvos: {validation_error}")

    finally:
        # Encerrar o Cliente Dask
        client.close()
        print("Cliente Dask encerrado.")

# Executar a Função Principal
if __name__ == "__main__":
    main()

In [7]:
# output_path_versioned = '../output_v20241222-104919'
# parquet_pattern = os.path.join(output_path_versioned, 'part.*.parquet')
# df_dask = dd.read_parquet(
#     parquet_pattern,
#     engine='pyarrow'
# )

In [8]:
# df_dask.head()